# Predicting Best Picture Winners & Nominees
*An Analysis by Sean Osier*

## Data Cleaning and Processing

In [627]:
# Import Dependencies
import pickle
import datetime
import dateutil.parser
import string

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.mpl_style = 'default'

import numpy as np
import re

# For display
import pprint
import matplotlib.pyplot as plt
%matplotlib inline

In [628]:
# Pickling functions
def pickle_it(data, filename, python_version=3):
    """
    In:
    data = the data you want to pickle (save)
    filename = file name where you want to save the data
    python_version = the python version where you will be opening the pickle file
    
    Out:
    Saves a pickle file with your data to to the filename you specify
    """
    with open(filename, "wb") as picklefile:
        pickle.dump(data, picklefile, protocol=python_version)

def load_pickle(filename):
    """
    In:
    filename = name of the pickle file you want to open (e.g "my_pickle.pkl")
    
    Out:
    Opens and returns the content of the picklefile to a variable of your choice
    """
    with open(filename, "rb") as picklefile: 
        return pickle.load(picklefile)

In [629]:
def process_IMDB_data(movie_data):
    """
    In:
    movie_data = list of lists of raw movie data scraped from IMDB index list pages
    
    Out:
    df = Pandas dataframe with the raw movie data cleaned and processed
    """
    headers = ["title", "year", "link", "user_rating_long", "user_rating_short",
                           "outline", "director", "starring", "genre", "pg_rating", "runtime"]
    df = pd.DataFrame(movie_data, columns=headers)
    df.drop_duplicates(inplace=True)
    
    # Some initial cleaning
    df["year"] = df["year"].replace("(2012 Documentary)", "(2012)")
    
    # Key
    df["key"] = df["title"] + " " + df["year"]
    
    # Year
    df["year"] = df["year"].apply(lambda x: x[1:-1])
    
    # Link
    df["link"] = "http://www.imdb.com" + df["link"]
    
    # User Rating
    df["user_rating_short"] = df["user_rating_short"].convert_objects(convert_numeric=True)
    
    # User Rating n-size
    df["user_rating_n"] = df["user_rating_long"].apply(lambda x: x.split()[4] if x != "" else "")
    df["user_rating_n"] = df["user_rating_n"].apply(lambda x: x[1:].replace(",", "") if x != "" else "")
    df["user_rating_n"] = df["user_rating_n"].convert_objects(convert_numeric=True)
    
    # Director(s)
    df["director"] = df["director"].apply(lambda x: x.split(", "))
    
    # Lead Actors
    df["starring"] = df["starring"].apply(lambda x: x.split(", "))
    
    # Genre
    df["genre"] = df["genre"].apply(lambda x: x.split(" | "))
    
    # Runtime
    df["runtime"] = df["runtime"].apply(lambda x: x.split()[0] if x != "" else "")
    df["runtime"] = df["runtime"].convert_objects(convert_numeric=True)
    
    # Removed unneed columns
    df = df[["key", "title", "year", "user_rating_short", "user_rating_n", "director", "starring", "genre", \
            "runtime", "pg_rating", "link"]]
    
    # More cleaning
    df = df[(df["year"] != "2015") & (df["year"] != "2016") & (df["year"] != "????") & (df["year"] != "???? ")]
    keep_criterion = df["runtime"].map(lambda x: ((x >= 45) and (x <= 360)) or (pd.isnull(x)))
    df = df[keep_criterion]
    
    return df

In [630]:
def process_nominee_and_winner_data(data):
    """
    In:
    data = list of lists of raw Best Picture winner and nominee data scraped from Wikipedia
    
    Out:
    df = Pandas dataframe with the raw Best Picture status data cleaned and processed
    """
    headers = ["title", "year", "status"]
    df = pd.DataFrame(data, columns=headers)
    df.drop_duplicates(inplace=True)
    
    # Initial cleaning
    df["title"] = df["title"].replace("The Godfather Part III", "The Godfather: Part III")
    df["title"] = df["title"].replace("Good Night, and Good Luck", "Good Night, and Good Luck.")
    df["title"] = df["title"].replace("Precious: Based on the Novel \"Push\" by Sapphire", "Precious")
    df["title"] = df["title"].replace("Extremely Loud and Incredibly Close", "Extremely Loud & Incredibly Close")
    df["title"] = df["title"].replace("Birdman or (The Unexpected Virtue of Ignorance)", 
                                      "Birdman: Or (The Unexpected Virtue of Ignorance)")
    
    # Key
    df["key"] = df["title"] + " (" + df["year"] + ")"
    
    # Status Year (Year Nominated For)
    df["status_year"] = df["year"]
    
    df = df[["key", "title", "year", "status", "status_year"]]
    
    # More cleaning
    df["key"] = df["key"].replace("Il Postino: The Postman (1995)", "Il Postino: The Postman (1994)")
    df["key"] = df["key"].replace("Life Is Beautiful (1998)", "Life Is Beautiful (1997)")
    df["key"] = df["key"].replace("Crash (2005)", "Crash (2004)")
    df["key"] = df["key"].replace("The Hurt Locker (2009)", "The Hurt Locker (2008)")
    
    return df

In [631]:
def extract_color(color_list):
    """
    In:
    color_list = raw list of colors scraped from individula IMDB movie pages
    
    Out:
    colors = cleaned and processed list of colors
    """
    colors = []
    for color in color_list:
        if ("Color" in color) or ("color" in color):
            colors.append("Color")
        if ("Black" in color) or ("White" in color) or ("B&W" in color):
            colors.append("Black and White")
    
    colors = list(set(colors))
    return colors

In [632]:
def eval_division_string(s):
    """
    In:
    s = mathmatical string with division (e.g. "12 / 24" or "1 / 2 / 3")
    
    Out:
    result = float of the what s evaluates to 
    """
    numbers = s.split("/")
    result = ""
    for i, n in enumerate(numbers):
        if i == 0:
            result = float(n)
        else:
            result /= float(n)
    return result

In [633]:
def remove_numbers(s):
    """
    In:
    s = a string
    
    Out:
    A string with all numbers removed
    """
    return "".join([c for c in s if c not in string.digits])

In [634]:
# Load exchanges rates data
exchange_rates = load_pickle("exchange_rates.pkl")

In [635]:
def convert_to_USD(n, currency):
    """
    In:
    n = an amount of money
    currency = the currency of n
    
    Out:
    n in US dollars ($)
    """
    return n*exchange_rates[currency]

In [637]:
def process_detailed_movie_data(data):
    """
    In:
    data = raw detailed movie data scraped from individual IMDB movie pages
    
    Out:
    df = Pandas dataframe with the raw detailed movie data processed and cleaned
    """
    headers = ["link", "release_date", "critic_rating", "critic_rating_n", "writer", "country", "language", \
            "budget", "opening_weekend_gross", "production_company", "sound_mix", "color", "aspect_ratio"]
    df = pd.DataFrame(data, columns=headers)
    
    # Release date
    df["release_date_details"] = df["release_date"].apply(lambda x: len(x.split()))
    default = datetime.datetime(1000, 1, 1)
    df["release_date_datetime"] = df["release_date"].apply(lambda x: dateutil.parser.parse(x, default=default))
    df["release_month"] = df[df["release_date_details"] >= 2]["release_date_datetime"].apply(lambda x: x.month)
    df["release_day_in_year"] = df[df["release_date_details"] >= 3]["release_date_datetime"].apply( \
        lambda x: x.timetuple().tm_yday)
    
    # Critic rating
    df["critic_rating"] = df["critic_rating"].convert_objects(convert_numeric=True)
    
    # Budget
    df["budget"] = df["budget"].apply(lambda x: x.replace(",", ""))
    df["budget_currency"] = df["budget"].apply(lambda x: remove_numbers(x.split()[0]) if x != "" else x)
    df["budget_currency"] = df["budget_currency"].replace("$", "USD")
    df["budget_currency"] = df["budget_currency"].replace("£", "GBP")
    df["budget_currency"] = df["budget_currency"].replace("€", "EUR")
    df["budget_currency"] = df["budget_currency"].replace("€.", "EUR")
    df["budget_USD"] = df["budget"].apply(lambda x: float(x[1:].split()[-1]) if x != "" else x)
    df["budget_USD"] = df.apply(lambda x: convert_to_USD(x["budget_USD"], x["budget_currency"]) 
                                if x["budget_USD"] != "" else "", axis=1)
    
    # Gross
    df["opening_weekend_gross"] = df["opening_weekend_gross"].apply(lambda x: x.replace(",", ""))
    df["opening_weekend_gross_currency"] = df["opening_weekend_gross"].apply(lambda x: remove_numbers(x.split()[0]) 
                                                                             if x != "" else x)
    df["opening_weekend_gross_currency"] = df["opening_weekend_gross_currency"].replace("$", "USD")
    df["opening_weekend_gross_currency"] = df["opening_weekend_gross_currency"].replace("£", "GBP")
    df["opening_weekend_gross_currency"] = df["opening_weekend_gross_currency"].replace("€", "EUR")
    df["opening_weekend_gross_currency"] = df["opening_weekend_gross_currency"].replace("€.", "EUR")
    df["opening_weekend_gross_USD"] = df["opening_weekend_gross"].apply(lambda x: float(x[1:].split()[-1]) \
                                                                        if x != "" else x)
    df["opening_weekend_gross_USD"] = df.apply(lambda x: convert_to_USD( \
                                               x["opening_weekend_gross_USD"], x["opening_weekend_gross_currency"]) \
                                               if (x["opening_weekend_gross_USD"] != "") and \
                                               (x["opening_weekend_gross_currency"] != "") else "", axis=1)
    
    # Color
    df["color"] = df["color"].apply(lambda x: extract_color(x))
    
    # Aspect ratio
    df["aspect_ratio"] = df["aspect_ratio"].apply(lambda x: x.replace("x", ":"))
    df["aspect_ratio"] = df["aspect_ratio"].apply(lambda x: "!!!!!" if (":" not in x) and (x != "") else x)
    df["aspect_ratio"] = df["aspect_ratio"].apply(lambda x: re.sub(r"[a-zA-Z]", "", x))
    df["aspect_ratio"] = df["aspect_ratio"].apply(lambda x: x.replace(",", "."))
    df["aspect_ratio"] = df["aspect_ratio"].apply(lambda x: x.replace("2:35", "2.35"))
    df["aspect_ratio"] = df["aspect_ratio"].apply(lambda x: x.replace(" : ", ":"))
    df["aspect_ratio"] = df["aspect_ratio"].apply(lambda x: x.replace("4: 3", "4:3"))
    df["aspect_ratio"] = df["aspect_ratio"].apply(lambda x: x.replace("16: 9", ":"))
    df["aspect_ratio"] = df["aspect_ratio"].apply(lambda x: x.split()[0] if x != "" else x)
    df["aspect_ratio"] = df["aspect_ratio"].apply(lambda x: x.strip(":"))
    df["aspect_ratio"] = df["aspect_ratio"].apply(lambda x: x.replace(":", "/"))
    df["aspect_ratio"] = df["aspect_ratio"].apply(lambda x: x.replace("!!!!!", "Other"))
    df["aspect_ratio"] = df["aspect_ratio"].apply(lambda x: eval_division_string(x) \
                                                  if (x != "" and x != "Other") else x)
    df["aspect_ratio"] = df["aspect_ratio"].convert_objects(convert_numeric=True)
    
    return df

In [638]:
def merge_in_nominees_and_wins(df, nom_win_df):
    """
    In:
    df = Pandas dataframe of cleaned and processed movie data from IMDB index list pages
    nom_win_df = Pandas dataframe of cleaned and processed Best Picture status data from Wikipedia
    
    Out:
    new_df = Merged pandas data frame now including both the original movie data and Best Picture status
    """
    nom_win_for_merge = nom_win_df[nom_win_df["year"].astype(int) >= 1990]
    nom_win_for_merge = nom_win_for_merge[["key", "status", "status_year"]]
    
    new_df = pd.merge(df, nom_win_for_merge, on="key", how="left")

    new_df["status"][10186] = np.nan
    new_df["status_year"][10186] = np.nan
    new_df["status"][57111] = np.nan
    new_df["status_year"][57111] = np.nan
    
    return new_df

In [639]:
def merge_in_detailed_movie_data(df, detailed_df):
    """
    In:
    df = Merged pandas data frame including both the original movie data and Best Picture status
    detailed_df = Pandas dataframe of cleaned and processed detailed movie data from IMDB individual movie pages
    
    Out:
    new_df = Merged pandas dataframe now including original movie data, detailed movie data, and Best Picture status
    """
    new_df = pd.merge(df, detailed_df, on="link", how="left")
    
    return new_df

In [640]:
# Load in raw scraped data
movie_data = load_pickle("movie_data.pkl")
nominees_and_winners_raw = load_pickle("nominees_and_winners.pkl")
detailed_movie_data = load_pickle("detailed_movie_data.pkl")

In [ ]:
# Process the data
df = process_IMDB_data(movie_data)
nom_win_df = process_nominee_and_winner_data(nominees_and_winners_raw)
detailed_df = process_detailed_movie_data(detailed_movie_data)

df = merge_in_nominees_and_wins(df, nom_win_df)
df = merge_in_detailed_movie_data(df, detailed_df)

In [ ]:
"""Get the list of links used to scrape the raw detailed movie data (if needed)"""
# links = df["link"].values
# pickle_it(links, "all_links.pkl")
!ls

In [636]:
def get_inflation_multipliers():
    """
    In:
    None
    
    Out:
    Returns a dictionary with multipliers to adjust for inflation for each year 1990 - 2014
    """
    
    inflation_rates = ["5.4", "4.2", "3", "3", "2.6", "2.8", "3", "2.3", "1.6", "2.2", "3.4", "2.8", "1.6", "2.3", \
                       "2.7", "3.4", "3.2", "2.8", "3.8", "-0.4", "1.6", "3.2", "2.1", "1.5", "1.6"]
    inflation_rates = sorted(list(zip(range(1990, 2015), inflation_rates)), reverse=True)
    inflation_rates = [(y, float(rate)/100 + 1) for y, rate in inflation_rates]
    
    inflation_multipliers = {}
    for year, rate in inflation_rates:
        inflation_multiple = 1
        for year, rate in inflation_rates[:(2015-year)]:
            inflation_multiple *= rate
        inflation_multipliers[year] = inflation_multiple
    
    return inflation_multipliers

inflation_multipliers = get_inflation_multipliers()

In [ ]:
def final_clean(df):
    """
    In:
    df = Merged pandas dataframe including original movie data, detailed movie data, and Best Picture status
    
    Out:
    df = Merged pandas dataframe with additional (final) cleaning and processing
    """
    # Parental Guidance (MPAA) Rating
    df = df[df["pg_rating"] != "APPROVED"]
    df["pg_rating"] = df["pg_rating"].replace("X", "NC_17")
    df["pg_rating"] = df["pg_rating"].replace("NOT_RATED", "UNRATED")
    df["pg_rating"] = df["pg_rating"].replace("", "UNRATED")

    # Status and Status Score
    df["status_score"] = df["status"]
    df["status_score"] = df["status_score"].replace("W", 10)
    df["status_score"] = df["status_score"].replace("N", 5)
    df["status_score"] = df["status_score"].replace(np.nan, 0)
    
    # Status Year
    df["status_year"].fillna(df["year"], inplace=True)
    
    # Number Nominees
    df["num_nominees"] = df["status_year"].apply(lambda x: "5" if int(x) <= 2008 else ">5")
    
    # Budget
    df["budget_USD_real"] = df.apply(lambda x: x["budget_USD"] * inflation_multipliers[int(x["year"])] \
                                     if x["budget_USD"] != "" else x["budget_USD"], axis=1)
    df["budget_USD_real"] = df["budget_USD_real"].convert_objects(convert_numeric=True)
    
    # Opening Weekend Gross
    df["opening_weekend_gross_USD_real"] = df.apply(lambda x: x["opening_weekend_gross_USD"] * \
                                                    inflation_multipliers[int(x["year"])] \
                                                    if x["opening_weekend_gross_USD"] != "" \
                                                    else x["opening_weekend_gross_USD"], axis=1)
    df["opening_weekend_gross_USD_real"] = df["opening_weekend_gross_USD_real"].convert_objects(convert_numeric=True)
    
    return df
    
df = final_clean(df)

In [646]:
# Loads lists of directors, actors, actresses, writers
directors, actors, actresses = load_pickle("directors_actors_actresses.pkl")
stars = actors + actresses
writers = load_pickle("writers.pkl")

# Make list of genre to use as possible regression features
genres = sorted(list(set([genre for genre_list in df["genre"].values for genre in genre_list]))[1:])
genres_to_exclude = ["News", "Talk-Show", "Game-Show", "Reality-TV", "Documentary", "Adult"]
genres = [genre for genre in genres if genre not in genres_to_exclude]

In [647]:
def make_hist_dict(df, column):
    """
    In:
    df = Merged pandas dataframe with final cleaning complete
    column = Data column to make histogram / counter dictionary for
    
    Out:
    d = Histogram / counter dictionary of number occurances of each key in the data set
    """
    d = {}
    for lst in df[column].values:
        for x in lst:
            d[x] = d.get(x, 0) + 1
    
    return d

In [648]:
# Make list of countries to use as possible regression features
country_hist = make_hist_dict(detailed_df, "country")
countries = list(country_hist.items())
countries = sorted(countries, key=lambda x: x[1])[-25:]  # Take top 25
countries = [x[0] for x in countries]

In [649]:
# Make list of countries to use as possible regression features
language_hist = make_hist_dict(detailed_df, "language")
languages = list(language_hist.items())
languages = sorted(languages, key=lambda x: x[1])[-25:]  # Take top 25
languages = [x[0] for x in languages]

In [650]:
def make_boolean_columns(df, old_column, new_columns):
    """
    In:
    df = Merged pandas dataframe with final cleaning complete
    old_column = column of interest in df with data points that are list of items (e.g. "Directors" or "Starring")
    new_columns = list of columns to make as a boolean column (e.g. for "Directors" this would be a list of 
                  individual directors)
    
    Out:
    df = Merged pandas dataframe with final cleaning complete and the desired boolean columns added
    """
    for column in new_columns:
        column_name = old_column + "_"
        column_name += column.lower().replace(" ", "_").replace("-", "_").replace(".", "")
        df[column_name] = df[old_column].apply(lambda x: 1 if column in x else 0)
        
    return df

In [651]:
# Make all the boolean columns
df = make_boolean_columns(df, "director", directors)
df = make_boolean_columns(df, "starring", stars)
df = make_boolean_columns(df, "genre", genres)
df = make_boolean_columns(df, "writer", writers)
df = make_boolean_columns(df, "country", countries)
df = make_boolean_columns(df, "language", languages)
df = make_boolean_columns(df, "color", ["Color", "Black and White"])

In [653]:
# Get rid of columns not needed regression
df = df.drop(["title", "year", "user_rating_n", "director", "starring", "genre", "link", "status", "release_date", \
              "critic_rating_n", "writer", "country", "language", "budget", "budget", "opening_weekend_gross", \
              "production_company", "sound_mix", "color", "release_date_details", "release_date_datetime", \
              "budget_currency", "budget_USD", "opening_weekend_gross_currency", "opening_weekend_gross_USD"], axis=1)

In [ ]:
"""Uncomment this when you want to save the final dataframe you will you use for regression"""
# pickle_it(df, "df_for_regression.pkl")
!ls